# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from pandas.api.types import CategoricalDtype
from collections import Counter

df=pd.read_csv('/datasets/data.csv')
print(df.info())

display(df.head(10))
print('------------   цели кредита ---------------')
print(df['purpose'].unique())
print('------------  количество детей ------------')
print(df['children'].unique())
print('------------ сем. положение ---------------')
print(df['family_status'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


------------   цели кредита ---------------
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим 

Вывод

Данные полученны, приступаем к анализу

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# #заменяем пропущенные значения дохода на медианные для каждого типа занятости
# #print(df['total_income'].sum())


# df['income_type']=df['income_type'].astype('category')                #преобразуем столбец с типами дохода в категорию, т.к. на следующем шаге мы будем создавать словарь на основе этих значений
# IncStr=df.groupby('income_type')['total_income'].median().round(2)    #делаем список вида [тип дохода],[медианное значение дохода]
# incdict=dict(IncStr)                                                  #преобразуем в словарь


# def nan_to_mean(str):
#         return(incdict[str])

# df['total_income']=df['total_income'].fillna(0)                       #в исходном датасете заменяем NULL на 0, мне так удобнее работать
# df['Nt']=df[df['total_income']==0]['income_type'].apply(nan_to_mean)  #создаем новыq столбец, в котором с троках где был NULL пищем среднее значение для этого типа дохода
# df['Nt']=df['Nt'].astype('float64')                                   #преобразуем тип, т.к. после предидущего шага он имеет тип категория
# df['Nt']=df['Nt'].fillna(0)                                           #заменяем в новом столбце NULL  на 0
# df['total_income']=df['total_income']+df['Nt']                        #Складываем два столбца



df.fillna(df.groupby(['income_type']).transform('mean'),inplace=True)

Вышенаписанный код ужасен по самым низким стандартам, но это единственное что я смог сделать работающим.

Есть очень лаконичный вариант:

df['total_income']=df['total_income'].fillna(df['income_type'].map(incdict))

но проблема в том, что он работает как-то неправильно.
Он работает, но не проходит вот такую проверку:

 Сумма total_income до изменений + Сумма столбца в который записанны средние значения = Сумма total_income после изменений

А мой ужасный код соответствует этому условию.

In [3]:
#заменяем неверные данные о детях
df.replace({'children': {-1 : 1, 20 : 2}},inplace=True)

#приводим значения столбца education к одному регистру
df['education']=df['education'].str.lower()

#заменяем пустые значения в стаже на 0
df['days_employed']=df['days_employed'].fillna(0)


Вывод


children         - значение -1 некорректно, привести к 1
                   значение 20 сомнительно, привести к 2 (предполагаем ошибку ввода)           
days_employed    - есть пропуски и отрицательные значения, для исправления не достаточно данных о методе сбора информации.                        Если бы был нужен анализ по этому параметру, то можно предположить, что отрицательные значения это                              результат внесения начального и конечного периода работы наоборот. Т.е. в форме сбора, данные в ячейках                        занесены наоборот. Это даст отрицательное значение стажа, при вычислении разницы дат. Но поскольку по этому                    параметру анализ не нужен, игнорируем.
dob_years        - есть 0,  но поскольку по этому параметру анализ не нужен, игнорируем.
education        - ошибок в данных нет,  есть написание в разных регистрах - привести к одному
education_id     - ошибок в данных нет
family_status    - ошибок в данных нет, различных написаний нет
famili_status_id - ошибок в данных нет
gender           - одно не достоверное значение, но поскольку по этому параметру анализ не нужен, игнорируем.
income_type      - ошибок в данных нет
debt             - ошибок в данных нет
total_income     - есть пустые значения, и поскольку по этому параметру ведется анализ, то пустые строки отбрасываются
purpose          - ошибок в данных нет, но есть различные написания, нужно приводить к стандартному виду

### Замена типа данных

In [4]:
df['days_employed']=df['days_employed'].astype('int64')
df['total_income']=df['total_income'].astype('int64')

Вывод

days_empoyed - меняем на целое, т.к. дробные значения не имеют смысла, разница дат приема и увольнения на работу - целое число
total_income - меняем на целое, т.к. копейки в анализе не нужны и только загромождают представление итогов

можно было поменять все int64 на int32, точности бы хватило, занимаемое место уменьшилось, но это имеет смысл для больших наборов данных

### Обработка дубликатов

In [5]:
i=df.duplicated().sum()
if i>0:
    print('Найденно строк-дубликатов: {}'.format(i))
    df=df.drop_duplicates().reset_index(drop=True)
else:
    print('Найденно строк-дубликатов: {}'.format(df.duplicated().sum()))
    
df.info()


Найденно строк-дубликатов: 71
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Вывод

есть дубли, удаляем

### Лемматизация

In [6]:
#посмотрим частоту с которой встречаются цели кредита
cd=Counter(df['purpose'])
sorted_cd = {k: cd[k] for k in sorted(cd)}
for el in sorted_cd.items():
    print(el)

('автомобили', 478)
('автомобиль', 494)
('высшее образование', 452)
('дополнительное образование', 460)
('жилье', 646)
('заняться высшим образованием', 496)
('заняться образованием', 408)
('на покупку автомобиля', 471)
('на покупку подержанного автомобиля', 478)
('на покупку своего автомобиля', 505)
('на проведение свадьбы', 768)
('недвижимость', 633)
('образование', 447)
('операции с жильем', 652)
('операции с коммерческой недвижимостью', 650)
('операции с недвижимостью', 675)
('операции со своей недвижимостью', 627)
('покупка жилой недвижимости', 606)
('покупка жилья', 646)
('покупка жилья для сдачи', 651)
('покупка жилья для семьи', 638)
('покупка коммерческой недвижимости', 661)
('покупка недвижимости', 621)
('покупка своего жилья', 620)
('получение высшего образования', 426)
('получение дополнительного образования', 446)
('получение образования', 442)
('приобретение автомобиля', 461)
('профильное образование', 436)
('ремонт жилью', 607)
('свадьба', 791)
('свой автомобиль', 478)
('

17.08.21

Ниже написан ужасный код, подсчитывающий число слов в лемматизированной форме из столбца цели кредита.

Логика работы:

1. получаем список уникальных фраз из столбца цели кредита с числом сколько раз такая фраза входит в датафрейм
2. берем каждую фразу
3. лемматизируем и получаем список слов из исходной фразы в лемматизированной форме
4. для каждого слова из списка полученного на предидущем шаге, записываем в промежуточный датафрейм строку, состоящую из этого    слова и числа вхождений этого слова в исходный датафрейм в составе этой фразы.
   Т.е. если фраза "покупка автомобиля" встречается 100 раз, то будет записанно две строки {'покупка','100'}    {'автомобиль','100'}
5. После это легко группировкой получить итоговые значения.

Почему я сделал так:

Я не могу, физически не могу применить операцию лемматизации непосредственно к исходному датафрейму. Это медленная операция, в исходном датафрейме больше 21000 строк и только 42 уникальных значения. Т.е. мне нужно выполнить медленную операцию всего 42 раза вместо 21000.

In [8]:
#здесь мы определяем частоту, с кторой встречаются леммы в столбце цели кредита
m = Mystem() 
testdf=pd.DataFrame(columns=['word','kolvo'])                          #датафрейм для работы

cd=Counter(df['purpose'])                                              #получаем список фраз с числом их упоминания в цели крадита
for dict_el in cd:                                                     #по всем фразам
    str=m.lemmatize(dict_el)                                           #получаем лемму всех слов в фразе
    for w in str:                                                      #для каждого слова в лемматизированной форме в фразе
        if w!=' ' and w!='\n':                                         #пропускаем пробелы и концы строк
            new_row = pd.Series(data={'word': w , 'kolvo': cd[dict_el]}) #формируем строку для вставки
            testdf=testdf.append(new_row,ignore_index=True)              #добавляем строку вида слово : число упоминаний в составе данной фразы в исходных данных
            
display(testdf.groupby('word')['kolvo'].sum().sort_values(ascending=False)) #сортировка по частоте

display(testdf.groupby(['word']).agg({'kolvo': ['sum']}))                   #сортировка по алфавиту


word
недвижимость      6351
покупка           5897
жилье             4460
автомобиль        4306
образование       4013
с                 2918
операция          2604
свадьба           2324
свой              2230
на                2222
строительство     1878
высокий           1374
получение         1314
коммерческий      1311
для               1289
жилой             1230
сделка             941
дополнительный     906
заниматься         904
проведение         768
сыграть            765
сдача              651
семья              638
собственный        635
со                 627
ремонт             607
подержанный        486
подержать          478
приобретение       461
профильный         436
Name: kolvo, dtype: int64

,kolvo
,sum
word,
автомобиль,4306
высокий,1374
для,1289
дополнительный,906
жилой,1230
жилье,4460
заниматься,904
коммерческий,1311


1. Очевидно, что "автомобили", "автомобиль", "на покупку автомобиля", "на покупку подержанного автомобиля", "на покупку своего автомобиля", "приобретение автомобиля", "свой автомобиль","сделка с автомобилем", "сделка с подержанным автомобилем" все означают фактически один объект кредитования - автомобиль. Если быть до конца дотошным, то можно было-бы разделить эту категорию на две: новый автомобиль и подержанный автомобиль. Но исходя из имеющихся данных это невозможно сделать, потому что для этого необходимо, чтобы в каждой цели явно было прописанно новый или подержанный, а этого нет. Поэтому представляется разумным, объеденить это в одну категорию.

2. Так же, все строки, в которых встречается слово "образование" представляется разумным объединить в одну категорию.

3. Со строками, где встречается слово "свадьба" в разных написания та-же ситуация: объединяем в одну категорию

4. С недвижимостью чуть сложнее. Есть несколько моментов. Все варианты со словом "жилье" или "недвижимость" можно объединить в одну катеорию, но с несколькими замечаниями:
 4.1. "покупка жилья для сдачи" - не является отдельной категорией, т.к.  для чего-бы физическое лицо не покупало объект       недвижимости с правовой точки зрения разницы нет. Это все равно покупка объекта недвижимости физическим лицом.
 4.2. "строительство недвижимости" в трех написаниях в принципе может быть выделенно в отдельную категорию, т.к. фактически нет объекта недвижимости, который мог бы быть залогом и деньги фактически выделяются на покупку стройматериалов и оплату услуг стройтелей.
 4.3."покупка коммерческой недвижимости" может быть выделен в отдельную категорию, но юридически значимым является только то, какое лицо, физическое или юридическое, становиться собственником. Т.к. у нас все клиенты - физические лица, то нет никакой разницы покупает оно помещение со статусом жилое или не жилое. 

In [10]:
m = Mystem() 

#функция для стандартизации написания целей кредита
def PurposeNorm(str):
    lemm=m.lemmatize(str)
    if 'автомобиль' in lemm:
        return 'автомобиль'
    elif 'жилье' in lemm or 'недвижимость' in lemm:
        if 'строительство' in lemm:
            return 'строительство дома'
        else:
            return 'недвижимость'
    elif 'образование' in lemm:
        return 'образование'
    elif 'свадьба' in lemm:
        return 'свадьба'
    else:
        return 'что-то другое'
    
#получаем список уникальных значений целей кредита из исходнон датафрейма
test=df['purpose'].unique()
#создаем вспомогательный датафрейм для анализа и прогоняем по нему вышенаписанную функцию
testdf=pd.DataFrame(test)
testdf[1]=testdf[0].apply(PurposeNorm)
#переименовываем столбцы, чисто для эстетики
testdf.set_axis(['before','after'],axis='columns', inplace=True)

In [11]:
#перебираем вспомогательный датафрейм и заменяем в основном датафрейме цели кредита на нормализованные
for i in range(len(testdf)):
    df.loc[:,'purpose'].replace(testdf.loc[i, 'before'],testdf.loc[i, 'after'],inplace=True)


**Вывод**

В колонке назначение кредита данные представленны в различных написания. Нужно привести их к единообразному, стандартизированному написанию.

Для этого:
1. собираем все уникальные значения в отдельный датафрейм
2. каждую строку полученного датафрейма прогоняем через лемматизатор, для приведения написания слов к лемме
3. по ключевым словам задаем категории.
4. теперь мы имеем датафрейм с двумя столбцами, в одном написание как в исходных данных, во втором стандартизированная категория
5. используя этот датафрейм, заменяем в исходном датафрейм данные о цели кредита на стандартизованные.
 5.1. для этого перебираем наш вспомогательный датафрейм, и для каждой строки из него заменяем в исходном датафрейм старое значение на новое.

Примечание.
 В принципе, можно было прогнать исходный датафрейм через лемматизатор, но это катастрофически медленно работает.

### Категоризация данных

In [12]:
#определяем границы по которым будем категоризировать доход
perc =[.25, .50, .75]
include =['int']
desc = df.describe(percentiles = perc, include = include)
print(desc['total_income'].round(2))


count      21454.00
mean      167431.58
std        98060.60
min        20667.00
25%       107623.00
50%       151887.00
75%       202417.00
max      2265604.00
Name: total_income, dtype: float64


In [13]:
#доход (<100.000 / 500.000 / 1.000.000. / 2.000.000 / >2.000.000)
def IncomeKat(inSum):
    if inSum <107623:
        return('низкий')
    elif inSum>=107623 and inSum<142594:
        return('ниже среднего')
    elif inSum>=142594 and inSum<195820:
        return('выше среднего')
    elif inSum>=195820:
        return('высокий')
    else:
        return('какя-то ошибка с доходом')

#семейное положение (одинокий / пара / одинокий с детьми / пара с детьми)
def FamKat(row):
    if row['children']==0:
        if row['family_status'] in['женат / замужем','гражданский брак']:
            return('пара без детей')
        else:
            return('одинокий без детей')
    else:
        if row['family_status'] in ['женат / замужем','гражданский брак']:
            return('пара с детьми')
        else:
            return('одинокий с детьми')
        
#заполняем данными столбцы для категоризации
df['Income_Kat']=df['total_income'].apply(IncomeKat)
df['Family_Kat']=df.apply(FamKat,axis=1)   


#преобразуем тип данных
df['purpose']=df['purpose'].astype('category')

#преобразуем тип данных и устанавливаем свою последовательность сортировки
df['Family_Kat']=df['Family_Kat'].astype('category')
#Fam_cats_to_order=['одинокий без детей','одинокий с детьми','пара без детей','пара с детьми']
#Fam_covered_type = CategoricalDtype(categories=Fam_cats_to_order,ordered=True)
#df['Family_Kat'] = df['Family_Kat'].cat.reorder_categories(Fam_cats_to_order, ordered=True)

#преобразуем тип данных и устанавливаем свою последовательность сортировки
df['Income_Kat']=df['Income_Kat'].astype('category')
Inc_cats_to_order=['низкий','ниже среднего','выше среднего','высокий']
Inc_covered_type = CategoricalDtype(categories=Inc_cats_to_order,ordered=True)
df['Income_Kat'] = df['Income_Kat'].cat.reorder_categories(Inc_cats_to_order, ordered=True)


**Вывод**

Я преобразовал в тип данных "категория" три столбца, цель кредита, доход и семейное положение с информацией о детях. 

Цель кредита - очевидно, они и так преведены к списку значений, категоризация тут сама проситься
Доход - категоризацию этих данных необходимо обсуждать с заказчиком в части границ разбиения. Без категоризации в итоговом             запросе будет >19000 строк, что очень затруднит анализ
Семейное положение и дети - я свел к 4 значениям, чисто из любопытства, в принципе в этом нет необходимости.


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [14]:
#посчитаем процент невозврата кредита в зависимости от детей
dfp=df.pivot_table(index=['Family_Kat'], columns='debt', values='children', aggfunc='count')
dfp.set_axis(['нет','есть'],axis='columns', inplace=True)
dfp['процент невозврата']=round((dfp['есть']/(dfp['есть']+dfp['нет'])*100),1)
print(dfp.sort_values(by='процент невозврата'))


                     нет  есть  процент невозврата
Family_Kat                                        
пара без детей      9453   745                 7.3
одинокий без детей  3575   318                 8.2
пара с детьми       5718   574                 9.1
одинокий с детьми    967   104                 9.7


In [15]:
testdf=df.loc[:,['children','debt']].copy()
testdf['flag']=df['children'].apply(lambda s : 'дети есть' if s>0 else 'детей нет' )
dfp=testdf.pivot_table(index=['flag'], columns='debt', values='children', aggfunc='count')
dfp.set_axis(['нет','есть'],axis='columns', inplace=True)
dfp['процент невозврата']=round((dfp['есть']/(dfp['есть']+dfp['нет'])*100),1)
print(dfp.sort_values(by='процент невозврата'))

             нет  есть  процент невозврата
flag                                      
детей нет  13028  1063                 7.5
дети есть   6685   678                 9.2


In [16]:
dfp=df.pivot_table(index=['family_status'], columns='debt', values='children', aggfunc='count')
dfp.set_axis(['нет','есть'],axis='columns', inplace=True)
dfp['процент невозврата']=round((dfp['есть']/(dfp['есть']+dfp['нет'])*100),1)
print(dfp.sort_values(by='процент невозврата'))

                         нет  есть  процент невозврата
family_status                                         
вдовец / вдова           896    63                 6.6
в разводе               1110    85                 7.1
женат / замужем        11408   931                 7.5
гражданский брак        3763   388                 9.3
Не женат / не замужем   2536   274                 9.8


**Вывод**

Прошлый раз я писал: 
Наличие детей снижает риск невозврата примерно на 2.3% для пар с детьми и на 2.1% для одиноких с детьми 
Это не верно. Я в функции распределения по категориям семейного положения допустил ошибку в условии. Я добавил еще один способ подсчета, только по столбцу с детьми и увидел, что данные не сходяться и тогда нашел ошибку в функции.

Новый вывод:
Наличие детей увеличивает риск невозврата на 1.7% (разброс в зависимости от семейного положения от 1.5 до 1.8)


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
dfp=df.pivot_table(index=['family_status'], columns='debt', values='children', aggfunc='count')
dfp.set_axis(['нет','есть'],axis='columns', inplace=True)
dfp['процент невозврата']=round((dfp['есть']/(dfp['есть']+dfp['нет'])*100),1)
print(dfp.sort_values(by='процент невозврата'))

                         нет  есть  процент невозврата
family_status                                         
вдовец / вдова           896    63                 6.6
в разводе               1110    85                 7.1
женат / замужем        11408   931                 7.5
гражданский брак        3763   388                 9.3
Не женат / не замужем   2536   274                 9.8


**Вывод**

Исходя из данных предидущего шага, пара, в общем случае, имеет чуть меньший риск невозврата кредита, чем одинокий. Примерно 2%.

Данные по конкретным статусам семейного положения дают нам более 4% в пользу пар потерявщих партнера по сравнению с полностью независимыми заемщиками.

Интересным является факт, что пары потерявщие партнера по тем или иным причинам (смерть,развод) имеют менщий риск невозврата кредита чем состоящие в официальном браке. Хотя разница столь невелика (0.4% - 0.9%) что находиться практически на грани статистической погрешности.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [18]:
dfp=df.pivot_table(index=['Income_Kat'],columns='debt',values='children', aggfunc='count')
dfp.set_axis(['нет','есть'],axis='columns', inplace=True)
dfp['процент невозврата']=round((dfp['есть']/(dfp['есть']+dfp['нет'])*100),1)
print(dfp.sort_values(by='процент невозврата'))

                нет  есть  процент невозврата
Income_Kat                                   
высокий        5452   413                 7.0
низкий         4937   427                 8.0
ниже среднего  4024   385                 8.7
выше среднего  5300   516                 8.9


**Вывод**

Самые надежные заемщики - богатые заемщики. Разница 1.6% в пользу богатых по сравнению с самыми бедными

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
dfp=df.pivot_table(index=['purpose'],columns='debt',values='children', aggfunc='count')
dfp.set_axis(['нет','есть'],axis='columns', inplace=True)
dfp['процент невозврата']=round((dfp['есть']/(dfp['есть']+dfp['нет'])*100),1)
print(dfp.sort_values(by='процент невозврата'))

                     нет  есть  процент невозврата
purpose                                           
недвижимость        8295   638                 7.1
строительство дома  1734   144                 7.7
свадьба             2138   186                 8.0
образование         3643   370                 9.2
автомобиль          3903   403                 9.4


**Вывод**

Получатели кредита на покупку недвижимости самые надежные, что и очевидно. Неожиданно высокий процент невозврата в части покупки автомобиля. Видимо автомобиль не самый нужный для жизни предмет, и при этом высоколиквидный. Продать автомобиль и подать на банкротство проще, чем сделать это с квартирой. 


## Шаг 4. Общий вывод

Пары в официальном браке с детьми, имеющие доход от 1млн и берущие кредит на покупку недвижимости - самые надежные клиенты. Что ни для кого не является открытием америки.

Самый ненадежный клиент - одинокий человек без детей, берущий кредит на покупку автомобиля. Что тоже не новость.

Наличие детей в общем случае, без рассмотрения семейного положения, повышает риск невозврата. В исходных данных очень много пар без детей, практически столько же, сколько всех остальных категорий в сумме. С учетом того, что все разницы очень небольшие, около 2%, это по видимому и приводит к противоположным результатам при группировки данных только по детям и по детям с учетом семейного положения.